# 参数管理

In [2]:
import torch
from torch import nn

In [3]:
net = nn.Sequential(nn.Linear(4, 8),
                        nn.ReLU(),
                        nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.1909],
        [-0.2270]], grad_fn=<AddmmBackward0>)

## 参数访问

In [4]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.2210, -0.1173, -0.2530,  0.3017, -0.1835,  0.0978,  0.2142, -0.1467]])), ('bias', tensor([-0.2431]))])


In [5]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2431], requires_grad=True)
tensor([-0.2431])


In [6]:
net[2].weight.grad == None

True

## 一次性访问所有参数

In [10]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [11]:
# 通过名字获取参数
net.state_dict()['2.bias'].data

tensor([-0.2431])

## 从嵌套块收集参数

In [12]:
def block1():
    return nn.Sequential(nn.Linear(4, 8),
                        nn.ReLU(),
                        nn.Linear(8, 4),
                        nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}',block1())

    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.4471],
        [0.4471]], grad_fn=<AddmmBackward0>)

In [13]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


## 内置初始化

In [14]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0116, -0.0140,  0.0189,  0.0028]), tensor(0.))

In [15]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0038, -0.0078, -0.0101, -0.0122]), tensor(0.))

## 对不同的块使用不同的初始化方法

In [16]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.5901, -0.6411, -0.6564, -0.3020])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


## 自定义初始化

In [18]:
def my_init(m):
    if type(m) == nn.Linear:
        print("INIT", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

INIT weight torch.Size([8, 4])
INIT weight torch.Size([1, 8])


tensor([[ 0.0000,  0.0000,  0.0000,  9.0504],
        [ 6.5102,  0.0000, -9.2514, -0.0000]], grad_fn=<SliceBackward0>)

In [20]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  2.0000,  2.0000, 11.0504])

# 参数绑定，参数共享

In [21]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8),
                    nn.ReLU(),
                    shared,
                    nn.ReLU(),
                    shared,
                    nn.ReLU(),
                    nn.Linear(8, 1))

net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
